In [1]:
import netCDF4 as nc
import numpy as np
import pandas as pd

# Replace 'path_to_your_file.nc' with the actual path to your NetCDF file
file_path = 'ssi_v02r01_yearly_s1610_e2020_c20210204.nc'
output_excel = 'SSI.xlsx'  # Output Excel file name

# Open the NetCDF file
dataset = nc.Dataset(file_path, mode='r')

# Assuming 'SSI' is the variable name for Spectral Solar Irradiance
ssi_var = dataset.variables['SSI']
ssi_data = ssi_var[:]  # Read all data
print(dataset.variables)
print(ssi_data.shape)

print(np.mean(ssi_data[:,2004]),ssi_data[1,2004])


wavelength_var = dataset.variables['wavelength']
wavelength_data = wavelength_var[:] 



wavelength_band_width_var = dataset.variables['Wavelength_Band_Width']
wavelength_band_width_data = wavelength_band_width_var[:] 


dataset.close()


ssi_average = np.mean(ssi_data,axis=0)
print(ssi_average)

ssi_df = pd.DataFrame({
    'Wavelength': wavelength_data,
    'Wavelength_Band_Width': wavelength_band_width_data,
    'SSI': ssi_average
})


ssi_df.to_excel(output_excel, index=False)

print(f"SSI data saved to {output_excel}")

{'SSI': <class 'netCDF4._netCDF4.Variable'>
float32 SSI(time, wavelength)
    long_name: NOAA Climate Data Record of Yearly Solar Spectral Irradiance (W m-2 nm-1)
    standard_name: solar_irradiance_per_unit_wavelength
    units: W m-2 nm-1
    cell_methods: time: mean
    ancillary_variables: SSI_UNC
    missing_value: -99.0
unlimited dimensions: time
current shape = (411, 3785)
filling on, default _FillValue of 9.969209968386869e+36 used, 'wavelength': <class 'netCDF4._netCDF4.Variable'>
float32 wavelength(wavelength)
    long_name: Wavelength grid center
    units: nm
    standard_name: radiation_wavelength
unlimited dimensions: 
current shape = (3785,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'Wavelength_Band_Width': <class 'netCDF4._netCDF4.Variable'>
float32 Wavelength_Band_Width(wavelength)
    long_name: Wavelength band width. Centered on wavelength.
    units: nm
unlimited dimensions: 
current shape = (3785,)
filling on, default _FillValue of 9.96920996838

In [5]:
import pandas as pd
import numpy as np 


band = pd.read_excel("band_information.xlsx")
SSI = pd.read_excel("SSI.xlsx")




def map_ssi_to_bands(band_info_df, ssi_df):
    band_info_df['SSI'] = None

    
    for index, row in band_info_df.iterrows():
        center_wavelength = row['Center Wavelength (nm)']
        band_width = row['Band Width (nm)']
        
        
        min_wavelength = center_wavelength - (band_width / 2)
        max_wavelength = center_wavelength + (band_width / 2)
        
        
        ssi_values = ssi_df[(ssi_df['Wavelength'] >= min_wavelength) & (ssi_df['Wavelength'] <= max_wavelength)]['SSI']
        
      
        if not ssi_values.empty:
            band_info_df.at[index, 'SSI'] = ssi_values.mean()

    return band_info_df


band_information_with_ssi = map_ssi_to_bands(band, SSI)


band_information_with_ssi.to_excel("Band_with_SSI_Information.xlsx",index=False)